In [ ]:
import tensorflow as tf
import keras
import numpy as np 
import pandas as np
import cv2

In [ ]:
#convert image into gray scale
def convert_img_gray(image):  
    grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    (thresh, b) = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)
    return grayImage
 


In [ ]:
#resize the image
def image_resize(x, width = None, height = None, inter = cv2.INTER_AREA):
    image = cv2.imread(x)
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    dim = (width,height)
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [ ]:
img_rows, img_cols = 28, 28
batch_size = 128
num_classes = 10
epochs = 15

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          validation_data=(x_test, y_test), callbacks=[TQDMNotebookCallback()])

In [ ]:
#save the model
from keras.models import model_from_json
from keras.models import load_model

In [ ]:
model_digit_recognizer_keras = model.to_json()
with open("model_digit_recognizer_num.json", "w") as json_file:
    json_file.write(model_digit_recognizer_keras)
model.save_weights("model_digit_recognizer_num.h5")


In [ ]:
json_file = open('model_digit_recognizer_num.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model_digit_recognizer_num.h5")
loaded_model.save('model_digit_recognizer_num.hdf5')
loaded_model=load_model('model_digit_recognizer_num.hdf5')

In [ ]:
#load the input image
img = image_resize('path to input image',28,28)
img_final= convert_img_gray(img)
m = cv2.subtract(255, img_final).reshape(28,28) 
t = m.reshape(1,28,28,1)

In [ ]:
result = loaded_model.predict_classes(t)

In [ ]:
print(result)

In [ ]:
cv2.imshow("result",m)
cv2.waitKey(5000)
cv2.destroyWindow("result")